<a href="https://colab.research.google.com/github/arnaldog12/Machine_Learning/blob/master/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

|  |  |
|-------------|-------|
| 🎓 **Aprendizado** | Supervisionado |
| 📋 **Tarefa** | Classificação ou Regressão |
| 🔧 **Normalização** | Não |
| ⭐ **Dificuldade** | Médio |

# ⚙️ 0. Dependências

In [15]:
import numpy as np
from collections import Counter

from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# 🔍 1. Introdução

# 🎲 2. Dados

## 🏷️ Classificação

In [16]:
x_clf, y_clf = make_classification(n_samples=100, n_features=10, n_classes=3, n_informative=8, random_state=42)
x_train_clf, x_test_clf, y_train_clf, y_test_clf = train_test_split(x_clf, y_clf, test_size=0.2, random_state=42)

print(x_train_clf.shape, y_train_clf.shape)
print(x_test_clf.shape, y_test_clf.shape)

(80, 10) (80,)
(20, 10) (20,)


## 📈 Regressão

In [17]:
x_reg, y_reg = make_regression(n_samples=100, n_features=10, n_targets=1, random_state=42)
x_train_reg, x_test_reg, y_train_reg, y_test_reg = train_test_split(x_reg, y_reg, test_size=0.2, random_state=42)

print(x_train_reg.shape, y_train_reg.shape)
print(x_test_reg.shape, y_test_reg.shape)

(80, 10) (80,)
(20, 10) (20,)


# 💻 3. Implementação

## Métricas de Impureza

In [18]:
def entropy(y):
    if len(y) == 0:
        return 0.0

    _, class_counts = np.unique(y, return_counts=True)
    proportions = class_counts / y.shape[0]
    return -np.sum(proportions * np.log2(proportions))


def gini(y):
    if len(y) == 0:
        return 0.0

    _, class_counts = np.unique(y, return_counts=True)
    proportions = class_counts / y.shape[0]
    return 1.0 - np.sum(proportions**2)


def mse(y: np.ndarray):
    if len(y) == 0:
        return 0.0

    mean_y = np.mean(y)
    return np.mean((y - mean_y) ** 2)

## Decision Tree

In [19]:
class Node:
    def __init__(self, parent=None):
        self.parent = parent
        self.left_child = None  # less than a value (regression) OR not equal to a category (classification)
        self.right_child = None  # otherwise
        self.impurity = None
        self.col_index = None
        self.value = None
        self.is_leaf = True
        self.output = None


class BaseTree:
    def __init__(self, criterion=gini, max_depth=None, min_samples_split=2):
        self.criterion = criterion
        self.max_depth = np.inf if max_depth is None else max_depth
        self.min_samples_split = min_samples_split
        self.root = Node()

    def fit(self, x, y):
        self._build_tree(self.root, x, y)

    def predict(self, x):
        return np.array([self._predict_recursive(sample, self.root) for sample in x])

    def _compute_output(self, y):
        raise NotImplementedError()

    def _build_tree(self, parent_node, x, y, depth=0):
        parent_node.col_index, parent_node.value, parent_node.impurity = self._find_best_split(x, y)
        if parent_node.impurity == 0 or len(y) < self.min_samples_split or depth > self.max_depth:
            parent_node.output = self._compute_output(y)
            return

        x_left, y_left, x_right, y_right = self._split_data(x, y, parent_node.col_index, parent_node.value)

        left_child = Node(parent=parent_node)
        self._build_tree(left_child, x_left, y_left, depth + 1)

        right_child = Node(parent=parent_node)
        self._build_tree(right_child, x_right, y_right, depth + 1)

        parent_node.is_leaf = False
        parent_node.left_child = left_child
        parent_node.right_child = right_child

    def _find_best_split(self, x, y):
        best_impurity, best_col, best_value = 0.0, None, None

        n_features = x.shape[1]
        for feature_idx in range(n_features):
            feature_values = np.unique(x[:, feature_idx])
            for value in feature_values:
                gain = self._information_gain(x, y, feature_idx, value)
                if gain > best_impurity:
                    best_impurity = gain
                    best_col = feature_idx
                    best_value = value

        return best_col, best_value, best_impurity

    def _information_gain(self, x, y, col_index, threshold):
        parent_impurity = self.criterion(y)

        _, y_left, _, y_right = self._split_data(x, y, col_index, threshold)
        if len(y_left) == 0 or len(y_right) == 0:
            return 0.0

        left_impurity = self.criterion(y_left)
        right_impurity = self.criterion(y_right)
        p = len(y_left) / y.shape[0]
        child_impurity = p * left_impurity + (1 - p) * right_impurity

        gain = parent_impurity - child_impurity
        return gain

    def _split_data(self, x, y, col_index, threshold):
        left_mask = x[:, col_index] < threshold
        right_mask = np.invert(left_mask)
        return x[left_mask], y[left_mask], x[right_mask], y[right_mask]

    def _predict_recursive(self, x, node):
        if node.is_leaf:
            return node.output

        right_condition = x[node.col_index] > node.value
        return self._predict_recursive(x, node.right_child if right_condition else node.left_child)


## Random Forest

In [20]:
class BaseRandomForest:
    def __init__(self, n_estimators=10, criterion=gini, max_depth=None, min_samples_split=2, random_state=None):
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []
        self.feature_indices = []
        self.tree_class = None

        if random_state is not None:
            np.random.seed(random_state)

    def fit(self, x, y):
        n_samples, n_features = x.shape
        max_features = n_features

        # Criar e treinar as árvores
        for i in range(self.n_estimators):
            # Bootstrap sampling
            indices = np.random.choice(n_samples, n_samples, replace=True)
            x_bootstrap = x[indices]
            y_bootstrap = y[indices]

            # Seleciona subconjunto de features
            feature_indices = np.random.choice(n_features, max_features, replace=False)
            x_subset = x_bootstrap[:, feature_indices]

            # Cria e treina a árvore
            tree = self.tree_class(
                criterion=self.criterion,
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
            )
            tree.fit(x_subset, y_bootstrap)

            self.trees.append(tree)
            self.feature_indices.append(feature_indices)

    def predict(self, x):
        raise NotImplementedError


## 🏷️ Classificador

In [21]:
class DTClassifier(BaseTree):
    def _compute_output(self, y):
        classes, class_counts = np.unique(y, return_counts=True)
        return classes[np.argmax(class_counts)]

In [22]:
class RFClassifier(BaseRandomForest):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tree_class = DTClassifier

    def predict(self, x):
        # Coleta previsões de todas as árvores
        all_predictions = []
        for i, tree in enumerate(self.trees):
            x_subset = x[:, self.feature_indices[i]]
            predictions = tree.predict(x_subset)
            all_predictions.append(predictions)

        # Votação majoritária
        all_predictions = np.array(all_predictions)
        final_predictions = []
        for sample_idx in range(x.shape[0]):
            votes = all_predictions[:, sample_idx]
            most_common = Counter(votes).most_common(1)[0][0]
            final_predictions.append(most_common)

        return np.array(final_predictions)

In [23]:
my_rf = RFClassifier(n_estimators=20, criterion=gini, max_depth=None, min_samples_split=2, random_state=42)
my_rf.fit(x_train_clf, y_train_clf)

y_pred = my_rf.predict(x_test_clf)
print(accuracy_score(y_test_clf, y_pred))

0.7


### Comparação com o Scikit-learn

In [24]:
rf_sk = RandomForestClassifier(
    n_estimators=20,
    criterion="gini",
    max_depth=None,
    min_samples_split=2,
    random_state=42,
    max_features=None,
    bootstrap=True,
)
rf_sk.fit(x_train_clf, y_train_clf)

y_pred = rf_sk.predict(x_test_clf)
print(accuracy_score(y_test_clf, y_pred))

0.7


## 📈 Regressor

In [25]:
class DTRegressor(BaseTree):
    def _compute_output(self, y):
        return np.mean(y)

In [26]:
class RFRegressor(BaseRandomForest):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tree_class = DTRegressor

    def predict(self, x):
        # Coletar previsões de todas as árvores
        all_predictions = []
        for i, tree in enumerate(self.trees):
            x_subset = x[:, self.feature_indices[i]]
            predictions = tree.predict(x_subset)
            all_predictions.append(predictions)

        # Média das previsões
        return np.mean(all_predictions, axis=0)

In [27]:
my_rf = RFRegressor(n_estimators=20, criterion=mse, max_depth=None, min_samples_split=2, random_state=42)
my_rf.fit(x_train_reg, y_train_reg)
error = mean_squared_error(y_test_reg, my_rf.predict(x_test_reg))
print(f"MSE: {error:.3f}")

MSE: 17575.695


### Comparação com o Scikit-learn

In [28]:
rf_sk = RandomForestRegressor(
    n_estimators=20,
    criterion="squared_error",
    max_depth=None,
    min_samples_split=2,
    random_state=42,
    max_features=None,
    bootstrap=True,
)

rf_sk.fit(x_train_reg, y_train_reg)
sk_error = mean_squared_error(y_test_reg, rf_sk.predict(x_test_reg))
print(f"MSE: {sk_error:.3f}")

MSE: 20536.678


# 💭 Considerações Finais

- Nossa implementação considera sempre `max_features = n_features`. Existem outras abordagens de seleção de features (ver [documentação do sklearn][doc-sk])

- A implementação do scikit-learn depende de um `random_state`. Logo, é difícil fazer todos os resultados baterem sempre.

**random_state**: Controls the randomness of the estimator. The features are always randomly permuted at each split, even if splitter is set to "best". When max_features < n_features, the algorithm will select max_features at random at each split before finding the best split among them. **But the best found split may vary across different runs, even if max_features=n_features**. That is the case, if the improvement of the criterion is identical for several splits and one split has to be selected at random. To obtain a deterministic behaviour during fitting, random_state has to be fixed to an integer. See Glossary for details.

[doc-sk]: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html